## Simplify Note Texts using GPT-4o

**Goal of this notebook:**  
Use AI to extract ONLY the important components of the Mimic-III NOTEEVENT TEXT field. Removing details such as admission dates and medical dosages will reduce the token count, and simplify the text _while_ keeping relevant context and the original sentences that would be lost if we were to use a parser such as [Azure Text Analytics for Health](./01_az_text_analytics.ipynb) or the [mednlp](https://github.com/plandes/mednlp) python package.

The extracted data will be used to...  
- Simplify the note text before attempting to code. This will improve calssification results

**Requirements**  
- Setup Azure Open AI Resource with GPT-4o deployment, ensure [.env](./.env.sample) file is populated up to date

In [19]:
from src.prepare_mimic_iii import transform_data
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
from textwrap import dedent

import pandas as pd
import os
import tiktoken

load_dotenv(find_dotenv(), override=True)

pd.set_option('display.max_colwidth', None)

#### Prepare Data

In [20]:
# Get medical coding data, take a small subsample

# df = transform_data("data/") # Only re-run if change in preparation logic
df = pd.read_csv("data/joined/dataset_single_notes_full.csv.gz").sample(5, replace=False,random_state=1234)
print(df.shape)
display(df.dtypes)
display(df.head(5))

(5, 3)


HADM_ID       int64
TEXT         object
ICD9_CODE    object
dtype: object

HADM_ID  \
1284    105949   
17283   180534   
8373    138964   
10370   148420   
13068   161146   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

#### Use AOAI to Simplify Note Text

In [21]:
aoai_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_BASE"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-01"
)

In [22]:
# Function to get token counts

def get_token_counts(text):
    encoding = tiktoken.get_encoding('cl100k_base')
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [23]:
# Funciton to build prompt

def build_prompt(note):
    sys = """
    Parse the following medical note. Return any sentences that relates to a diagnosis. Ignore other information such as patient name, dates, medicine types, dosage amounts, prior medical history, prior treamtment, etc.
    DO NOT ADD ANY INFORMATION TO THE NOTE. ONLY RETURN THE RELEVANT SENTENCES. 
    """
    prompt = f"{note}"

    return (sys, prompt)

In [24]:
def aoai_extract(note):
    sys, prompt = build_prompt(note)
    response = aoai_client.chat.completions.create(
        model=os.getenv("AOAI_MAIN_DEPLOYMENT_NAME"), # model = "deployment_name".
        messages=[
            {"role": "system", "content": dedent(sys)},
            {"role": "user", "content": dedent(prompt)}
        ],
    )

    return response.choices[0].message.content


In [25]:
df["AOAI_EX"] = df["TEXT"].apply(lambda x: aoai_extract(x))

INFO:httpx:HTTP Request: POST https://medcode-aoai-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


In [26]:
df["AOAI_TOKENS"] = df["AOAI_EX"].apply(lambda x: get_token_counts(x))
df["TEXT_TOKENS"] = df["TEXT"].apply(lambda x: get_token_counts(x))

#### Examine Results

In [27]:
# Avg token count difference - How much efficiency is gained?
print(f"Average Original Text tokens: {df['TEXT_TOKENS'].mean()}")
print(f"Average AOAI Extracted tokens: {df['AOAI_TOKENS'].mean()}")
print(f"Average token count difference: {(df['TEXT_TOKENS'] - df['AOAI_TOKENS']).mean()}")

Average Original Text tokens: 3766.0
Average AOAI Extracted tokens: 279.8
Average token count difference: 3486.2


In [28]:
# Text analysis - Is any information lost?
sub_df=df[['ICD9_CODE','TEXT','AOAI_EX']]

In [29]:
# Example 0
display(sub_df.iloc[[0]])

ICD9_CODE  \
1284  ['805', '860', '851', '518', '486', '808', '900', '807', '998']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Analysis:  
- TODO

---


In [30]:
# Example 1
display(sub_df.iloc[[1]])

ICD9_CODE  \
17283  ['995', '595', '590', '620', '362', '401', '285', '591', '250', '038']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

Analysis:  
- TODO

---

In [31]:
# Example 2
display(sub_df.iloc[[2]])

ICD9_CODE  \
8373  ['787', '008', '276', '041', '348', '531', '403', '434', '585', '274', '530', '438', '599', '250', '790', '263', '532', '311']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Analysis:  
- TODO...

---

In [32]:
# Example 3
display(sub_df.iloc[[3]])

,ICD9_CODE,TEXT,AOAI_EX
10370,"['532', '518', '401', '286', '263', '785', '276']","[""Admission Date: [**2179-9-20**] Discharge Date: [**2179-10-1**]\n\nDate of Birth: Sex:\n\nService: GENERAL\n\nHISTORY OF THE PRESENT ILLNESS: This is a 72-year-old woman\nwho presented with three days of melena and one week of\nincreasing weakness. Because of the complaints of chest\npain, her son brought her to our emergency room for\nevaluation. She ultimately proved to have a perforated\nduodenal ulcer.\n\nPAST MEDICAL HISTORY:\n1. History of alcohol abuse with alcoholic cirrhosis.\n2. Arthritis.\n3. Hypertension.\n4. History of nephrolithiasis.\n\nALLERGIES: None.\n\nMEDICATIONS:\n1. Naprosyn.\n2. Lopressor.\n3. Procardia.\n\nPHYSICAL EXAMINATION: The patient was afebrile with a blood\npressure of 95/40. She was awake, but sleepy. She appeared\nuncomfortable. The chest was clear. Abdomen was obese with\nvoluntary guarding and tenderness in the upper abdomen.\nRectal examination showed guaiac-positive stool.\n\nLABORATORY DATA: Data revealed the following: The admission\nWBC was 21,000 with hematocrit of 33.5. The bicarbonate was\n18, BUN 39, creatinine 4.3, ALT 63, AST 74, alkaline\nphosphatase 569, total bilirubin 4.2 with the direct\ncomponent being 3.8. Amylase 35, and lipase 132. The\nurinalysis was unremarkable. Chest film showed free air\nunder the right hemidiaphragm.\n\nHOSPITAL COURSE: Contrast upper GI was obtained showing a\nperforation of the first portion of the duodenum. The\npatient was aggressively fluid resuscitated. She was taken\nto the operating room that evening for laparotomy. At that\ntime she was found to have a perforation in the anterior\nduodenal bulb. We closed this perforation primarily and\nplaced [**Initials (NamePattern4) **] [**Last Name (NamePattern4) **] patch. Postoperatively, the patient did\npoorly. She was persistently septic in the Intensive Care\nUnit with a very large fluid requirement. She essentially\ndeveloped progressive ascites and findings of end-stage liver\ndisease. Bilirubin remained elevated. She required\nprolonged ventilatory support. She developed progressive\nanuric renal failure. Consultation with the Renal Medicine\nService was obtained. She was maintained on broad-spectrum\nantibiotics and IV nutrition. She was extremely somnolent.\nShe developed progressive severe ARDS. Fungal coverage was\nadded to her drug regimen. Intensive consultations were had\nwith the son and with the [**Hospital 228**] healthcare proxy.\nUltimately, decision was made to withdrawal support due to\nthe patient's lack of progress and wishes that had been\nexpressed by her, prior to surgery. Accordingly, the patient\nwas made comfort-measures only and expired on [**10-1**].\n\nDISPOSITION: Deceased.\n\nCONDITION ON DISCHARGE: Deceased.\n\nDISCHARGE DIAGNOSIS: Cirrhotic liver failure with ascites,\nsepsis, severe acute respiratory distress syndrome,\ndisseminated intravascular coagulation, perforated duodenal\nulcer.\n\n\n\n\n [**First Name11 (Name Pattern1) **] [**Last Name (NamePattern4) 520**], M.D. [**MD Number(1) 521**]\n\nDictated By:[**Last Name (NamePattern4) 1779**]\nMEDQUIST36\n\nD: [**2180-3-15**] 08:37\nT: [**2180-3-15**] 15:21\nJOB#: [**Job Number 13819**]\n""]","- She ultimately proved to have a perforated duodenal ulcer.\n- Contrast upper GI was obtained showing a perforation of the first portion of the duodenum.\n- She essentially developed progressive ascites and findings of end-stage liver disease.\n- She developed progressive anuric renal failure.\n- She developed progressive severe ARDS.\n- DISCHARGE DIAGNOSIS: Cirrhotic liver failure with ascites, sepsis, severe acute respiratory distress syndrome, disseminated intravascular coagulation, perforated duodenal ulcer."


Analysis:  
- TODO

---

In [33]:
# Example 4
display(sub_df.iloc[[4]])

ICD9_CODE  \
13068  ['518', '584', '296', '401', '458', '969', '535', '285', '599', '507', '794', '728', '293', '276', '965']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

Analysis:  
- TODO

---